# Chapter 9: Iterators

Here we present all code from this chapter, so that you can reproduce results by your own.

We start by importing all general modules

In [1]:
from scipy import *
import itertools

## Generators

### Example: Odd numbers

A generator for odd numbers smaller than $n$ could be defined like this:

In [2]:
def odd_numbers(n):
    "generator for odd numbers less than n"
    for k in range(n):
        if k % 2 == 1:
            yield k

Then you can use it as:

In [3]:
g = odd_numbers(10)
for k in g:
    print(k,end=' ') # do something with k

1 3 5 7 9 

or even as:

In [4]:
for k in odd_numbers(10):
    print(k,end=' ') # do something with k

1 3 5 7 9 

## Iterators are Disposable

Note that an iterable object is able to create new iterators as many time as necessary.
Let us examine the case of a list:

In [5]:
L = ['a', 'b', 'c']
iterator = iter(L)
list(iterator) # ['a', 'b', 'c']
list(iterator) # [] empty list, because the iterator is exhausted

new_iterator = iter(L) # new iterator, ready to be used
list(new_iterator) # ['a', 'b', 'c']

['a', 'b', 'c']

## Iterator Tools

In [6]:
A = ['a', 'b', 'c']
for iteration, x in enumerate(A):
    print(iteration, x)


0 a
1 b
2 c


In [7]:
A = [0, 1, 2]
for elt in reversed(A):
    print(elt)

2
1
0


In [8]:
for iteration in itertools.count():
    if iteration > 5:
        break # without this, the loop goes on forever
    print("integer {}".format(iteration))

integer 0
integer 1
integer 2
integer 3
integer 4
integer 5


Let us find some odd numbers by combining `islice` with an infinite generator. First we modify the 
generator for odd numbers so that it becomes an infinite generator:

In [9]:
def odd_numbers():
    k=-1
    while True:
        k+=1
        if k%2==1:
            yield k

Then, we use it with `islice` to get a list of some odd numbers:

In [10]:
list(itertools.islice(odd_numbers(),10,30,8))

[21, 37, 53]

## Generators of Recursive Sequences

### Fibonacci Numbers

In [11]:
def fibonacci(u0, u1):
    """
    Infinite generator of the Fibonacci sequence.
    """
    yield u0
    yield u1
    while True:
        u0, u1 = u1, u0+u1
        yield u1

In [12]:
list(itertools.islice(fibonacci(0, 1), 15))

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377]

## Example: Arithmetic Geometric Mean

In [13]:
def arithmetic_geometric_mean(a, b):
    """
    Generator for the arithmetic and geometric mean
    a, b initial values
    """ 
    while True:	# infinite loop
        a, b = (a+b)/2, sqrt(a*b)
        yield a, b

In [14]:
from itertools import islice
def elliptic_integral(k, tolerance=1e-5, maxiter=100):
    """
    Compute an elliptic integral of the first kind.
    """
    a_0, b_0 = 1., sqrt(1-k**2)
    for a, b in islice(arithmetic_geometric_mean(a_0, b_0), maxiter):
        if abs(a-b) < tolerance:
            return pi/(2*a)
    else:
        raise Exception("Algorithm did not converge")

In [15]:
def pendulum_period(L, theta, g=9.81):
    return 4*sqrt(L/g)*elliptic_integral(sin(theta/2))

In [16]:
L=1.5
theta=pi/4
print('Period of pendulum with length {} and initial angle theta {} degree is'.format(L,theta*180/(2*pi)),
                                                                                 pendulum_period(L,theta))

Period of pendulum with length 1.5 and initial angle theta 22.5 degree is 2.5551310824


## Convergence Acceleration

In [17]:
def Euler_accelerate(sequence):
    """
    Accelerate the iterator in the variable `sequence`.
    """
    s0 = next(sequence) # Si
    s1 = next(sequence) # Si+1
    s2 = next(sequence) # Si+2
    while True:
        yield s0 - ((s1 - s0)**2)/(s2 - 2*s1 + s0)
        s0, s1, s2 = s1, s2, next(sequence)

In [18]:
def pi_series():
    sum = 0.
    j = 1
    for i in itertools.cycle([1, -1]):
        yield sum
        sum += i/j
        j += 2

In [19]:
N=10
list(itertools.islice(Euler_accelerate(pi_series()), N))

[0.75,
 0.7916666666666667,
 0.7833333333333334,
 0.7863095238095239,
 0.784920634920635,
 0.7856782106782109,
 0.7852203352203354,
 0.7855179542679545,
 0.7853137059019414,
 0.7854599047323508]

## Storing generated values when an exception is raised

In [20]:
import itertools
def power_sequence(u0):
    u = u0
    while True:
        yield u
        u = u**2

In [21]:
generator = power_sequence(2.)
L = []
for iteration in range(20):
    try:
        L.append(next(generator))
    except Exception:
        break
print(L)

[2.0, 4.0, 16.0, 256.0, 65536.0, 4294967296.0, 1.8446744073709552e+19, 3.402823669209385e+38, 1.157920892373162e+77, 1.3407807929942597e+154]


## When Iterators Behave as Lists

### Generator expressions

In [22]:
g = (n for n in range(1000) if not n % 100)

In [23]:
print(list(g))

[0, 100, 200, 300, 400, 500, 600, 700, 800, 900]


In [24]:
sum(n for n in range(1000) if not n % 100) 

4500

#### $\zeta$-function

In [25]:
def generate_zeta(s):
    for n in itertools.count(1):
        yield 1/n**s

def zeta(N, s):
    return sum(x for x in itertools.islice(generate_zeta(s), N))

zeta(1000,2)

1.6439345666815615

## Iterator Objects

### ODE store

In [26]:
class OdeStore:
    """Class to store results of ode computations"""
    def __init__(self, data):
        "data is a list of the form [[t0, u0], [t1, u1],...]"
        self.data = data
    def __iter__(self):
        "By default, we iterate on the values u0, u1,..."
        for t, u in self.data:
            yield u

store = OdeStore([[0, 1], [0.1, 1.1], [0.2, 1.3]])
for u in store:
    print(u)
list(store) 

1
1.1
1.3


[1, 1.1, 1.3]